# Different long-term thermal comfort indices for continuous monitoring

In [1]:
from temperature_range_percentage import *

In [2]:
temp_range?

Signature: temp_range(a, b, l, u, f)
Docstring:
Calculate the percentage of occupied time outside a temeprature range.
The data file type should be CSV.
The first column of the CSV file should be time.
The second column of the CSV file should be temperature.

Parameters
----------
a : int
    The start time (24-hour clock) of normal office hours during weekdays
b : int
    The end time (24-hour clock) of normal office hours during weekdays
l : float
    lower bound of the tempearture range, with same units of the data
u : float
    upper bound of the temperature range, with same units of the data
f : string
    file path of the CSV dataset


Returns
----------
p : float
    percentage of the time
File:      ~/Documents/GitHub/mortardata_comfort_evaluation/temperature_range_percentage.py
Type:      function


## No.1: Percentage of occupied time outside a temeprature range

In [3]:
Two_East = '/Users/ruijisun/Documents/20210727 Zone Temperature Data/DBC/Field Bus1.BrowerRadiant2East.ZN2-T.csv'

print("{:.0%}".format(temp_range(9, 17, 72, 81, Two_East)))

82%


In [4]:
Three_West= '/Users/ruijisun/Documents/20210727 Zone Temperature Data/DBC/Field Bus1.BrowerRadiant3West.ZN1-T.csv'
print("{:.0%}".format(temp_range(9, 17, 72, 81, Three_West)))

38%


In [5]:
Four_East = '/Users/ruijisun/Documents/20210727 Zone Temperature Data/DBC/Field Bus1.BrowerRadiant4East.ZN2-T.csv'
print("{:.0%}".format(temp_range(9, 17, 72, 81, Four_East)))

40%


## No.2: Mean Temperature 

In [6]:
from temperature_mean import *

In [7]:
temp_mean(9, 17, Two_East)

67.82025979822039

In [8]:
temp_mean(9, 17, Three_West)

72.63266124303229

In [9]:
temp_mean(9, 17, Four_East)

71.57280375565651

## No.3: Temperature Variance 

In [10]:
from temperature_variance import *

In [11]:
temp_var(9, 17, Three_West)

25.258746009585497

In [12]:
temp_var(9, 17, Four_East)

32.218171462457214

## No.4: Percentage of occupied days that daily temperature range are out of the threshold

In [14]:
from temperature_daily_outlier import *

In [15]:
print("{:.0%}".format(temp_daily(9, 17, 0.8, Two_East)))

21%


In [16]:
print("{:.0%}".format(temp_daily(9, 17, 0.8, Three_West)))

21%


In [17]:
print("{:.0%}".format(temp_daily(9, 17, 0.8, Four_East)))

21%


In [21]:
df = pd.read_csv(Two_East)
time = df.columns[0]
temp = df.columns[1]
# get the date from the time for groupby function later on
df['date'] = pd.to_datetime(df[time]).dt.date
df['hour'] = pd.to_datetime(df[time]).dt.hour
df['weekdays'] = pd.to_datetime(df[time]).dt.dayofweek
# create a new occupied dataframe from 9 am to 5 pm at weekdays
df_occ = df[(df['hour'] >= 9) & (df['hour'] < 17) &
            (df['weekdays'] >= 0) & (df['weekdays'] <= 4)]
# calculate daily temperature range by max minus min
df_max = df_occ.groupby(['date']).max()
df_min = df_occ.groupby(['date']).min()
df_range = df_max[temp] - df_min[temp]

In [22]:
df_occ

,datetime,/FS4/NAE00806603BECA/3001087/NAE00806603BECA/Field Bus1.BrowerRadiant2East.ZN2-T,date,hour,weekdays
0,2020-12-01 10:05:00,70.299370,2020-12-01,10,1
1,2020-12-01 10:10:00,70.299370,2020-12-01,10,1
2,2020-12-01 10:15:00,70.299370,2020-12-01,10,1
3,2020-12-01 10:20:00,70.299370,2020-12-01,10,1
4,2020-12-01 10:25:00,70.502136,2020-12-01,10,1
...,...,...,...,...,...
34556,2021-03-31 10:45:00,67.608704,2021-03-31,10,2
34557,2021-03-31 10:50:00,67.608704,2021-03-31,10,2
34558,2021-03-31 10:55:00,67.820107,2021-03-31,10,2
34559,2021-03-31 11:00:00,67.820107,2021-03-31,11,2


In [23]:
df_range

date
2020-12-01    1.812111
2020-12-02    2.847229
2020-12-03    2.847633
2020-12-04    2.434929
2020-12-07    3.247620
                ...   
2021-03-25    1.608479
2021-03-26    1.810432
2021-03-29    1.213501
2021-03-30    1.628166
2021-03-31    1.025452
Name: /FS4/NAE00806603BECA/3001087/NAE00806603BECA/Field Bus1.BrowerRadiant2East.ZN2-T, Length: 87, dtype: float64

In [25]:
df_range.quantile(0.8)

2.8231246948242186

In [27]:
t = df_range.quantile(0.8)
df_out = df_range[(df_range > t)]
# Calculate the percentage of occupied time outside the threshold
p = len(df_out) / len(df_max)

In [28]:
p

0.20689655172413793